# kNN Model

For this project, I have tested the performance of kNN with k = 1, 3, 5 on the training data set. It turns out it is not working really well on this data set, as the auc score is very close to 0.5 even with increasing of k, which means the predictions we get from the model is not much better than guessing. This is due to the imbalanced classes of target, this dataset has more than 96% of zeros in the targets.

In my experiment I have tried the bagging and resampling strategies to impprove my kNN model, however kNN model is not very suitable for bagging and boosting, as it is too stable when k is bigger than 1, I have tried bagging with 1-NN model however it doesn't improve much. I have compared over-sampling and under-sampling, I found out under-sampling works a little bit better than oversampling, I think that this is because the size of our data set is huge, so lossing information is better than adding noise to our modelling.

In this file, I will include my result of kNN models trainned on full train dataset and on the under-sampling train dataset. I will choose the one trainned on the under-sampling dataset as my submission model.

In [5]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler #for standardizing data
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import  roc_curve,auc, accuracy_score, confusion_matrix
from math import ceil
from sklearn.ensemble import BaggingClassifier,RandomForestClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split,cross_val_score,cross_val_predict
from numba import jit
import time

In [2]:
@jit
def gini(y_true, y_prob):
    y_true = np.asarray(y_true)
    y_true = y_true[np.argsort(y_prob)]
    ntrue = 0
    gini = 0
    delta = 0
    n = len(y_true)
    for i in range(n-1, -1, -1):
        y_i = y_true[i]
        ntrue += y_i
        gini += y_i * delta
        delta += 1 - y_i
    gini = 1 - 2 * gini / (ntrue * (n - ntrue))
    return gini

### Import the train and test data sets.

In [4]:
train = pd.read_csv('C:/Users/nd19620/OneDrive - University of Bristol/DST/DST_Assessment1/Xinyue Guan/Data/imputetrain.csv')
test = pd.read_csv('C:/Users/nd19620/OneDrive - University of Bristol/DST/DST_Assessment1/Xinyue Guan/Data/New_test.csv')

In [10]:
(train['target']==0).sum()/train.shape[0] #percentage of zeros in our labels

0.9637216095553629

### Performance of Basic kNN Models

In [ ]:
y_train = train['target'].values
id_train = train['id'].values
X_train = train.drop(['target', 'id'], axis=1)

id_test = test['id'].values
X_test = test.drop(['target','id'], axis=1)
y_test = test['target'].values

X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)
y_train1=y_train.ravel()

In [ ]:
scaler = StandardScaler()
scaled_train0= scaler.fit_transform(X_train)
X_train = pd.DataFrame(scaled_train0, columns=X_train.columns)
scaled_test0 = scaler.fit_transform(X_test)
X_test = pd.DataFrame(scaled_test0, columns=X_test.columns)

In [ ]:
cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)
pred_test_full = np.zeros(ceil(4*len(id_train)/5))
knn_model1 = KNeighborsClassifier(n_neighbors=1)
knn_model3 = KNeighborsClassifier(n_neighbors=3)
knn_model5 = KNeighborsClassifier(n_neighbors=5)

The scores of 5-fold cross validation takes a long time to run, so I have plotted my result below.

In [ ]:
t0 = time.time()
predicts = cross_val_predict(knn_model1, X_train, y_train1, cv=cv, n_jobs=-1)
auc_scores1 = []
gini_scores1 = []
for i in range(5):
    fpr, tpr, threshold = roc_curve(y_test,predicts[,i])
    auc_scores1.append(auc(fpr, tpr))
    gini_scores1.append(gini(y_test,predicts[,i]))
auc_scores1.to_csv('1NN_5-fold_cv_auc_scores.csv')
gini_scores1.to_csv('1NN_5-fold_cv_gini_scores.csv')
t1 = time.time()

runtime = t1-t0
print(runtime)

In [ ]:
scores3 = cross_val_score(knn_model3, X_train, y_train1, scoring='roc_auc', cv=cv, n_jobs=-1)

In [ ]:
scores5 = cross_val_score(knn_model5, X_train, y_train1, scoring='roc_auc', cv=cv, n_jobs=-1)

### Under Sampling for Train Set
After experimenting wiht

In [4]:
count_class_0, count_class_1 = train.target.value_counts()
df_class_0 = train[train['target'] == 0]
df_class_1 = train[train['target'] == 1]

In [ ]:
df_class_0_under = df_class_0.sample(count_class_1)
train_under = pd.concat([df_class_0_under, df_class_1], axis=0)